In [3]:
import csv

In [4]:
from bs4 import BeautifulSoup

In [5]:
import requests as uReq

In [6]:
import re

In [7]:
def dividend_return(dividend_check, trading_price):
    
    if ',' in dividend_check or len(dividend_check.split(' ')) == 3 or len(dividend_check) > 8:
        dividends = dividend_check.split(',') if ',' in dividend_check else dividend_check.split(' ', 1)   #stores dividend paid in both cash and bonus shares
        
        dividend = dividend_in_bonus_cash(dividends, trading_price)
    
    elif 'B' in dividend_check  and ',' not in dividend_check: 
        dividend = dividend_in_bonus(dividend_check, trading_price)
    
    elif '-' in dividend_check:
        dividend = dividend_check   #when dividend isn't mentioned on the website
    
    elif 'B' not in dividend_check and '-' not in dividend_check and ',' not in dividend_check:
        dividend = dividend_cash(dividend_check)   #Dividend paid in cash
    
    return dividend

In [8]:
('.00 ' in '12.00 16B' or '12' in '11') and ('13' in '12')

False

In [9]:
def dividend_in_bonus_cash(cash_and_bonus_dividends, market_price):
    """This function calculates total dividend from cash and bonus issues"""
    
    for dividend in cash_and_bonus_dividends:
        if 'B' not in dividend:
            cash_dividend = float(dividend.replace('%', '').strip())
            
        elif 'B' in dividend:
            bonus_dividend = float(dividend.strip().replace('%', '').replace('B',''))
    
    if '-' not in market_price:
        return round(((cash_dividend / 100) * 10) + ((bonus_dividend / 100) * float(market_price)), 2)
    
    elif '-' in market_price:
        return [cash_dividend, bonus_dividend]

In [10]:
dividend_in_bonus_cash('2.50, 2.50%B'.split(','), '-')

[2.5, 2.5]

In [11]:
def dividend_in_bonus(bonus_dividend, market_price):
    '''Calculates the amount of last bonus share dividend'''
    dividend = float(bonus_dividend.strip().replace('%', '').replace('%','').replace('B',''))
    
    if '-' not in market_price:
        return round(((dividend / 100) * float(market_price)), 2)
    
    elif '-' in market_price:
        return str(dividend) + '%B'

In [12]:
def dividend_cash(cash_dividend):
    return round((float(cash_dividend.replace('%','')) / 100) * 10 , 2)

In [13]:
def calculate_eps(pe, market_price):
    '''Receives P/E and Trading price as arguments and returns eps'''
    if '-' in pe or 'n/a' in pe or '-' in market_price:
        eps = pe
    else:
        eps = round(float(market_price) / float(pe), 2)
    
    return eps

In [14]:
def calculate_navps(navps):
    if '-' in navps or 'n/a' in navps:
        return navps
    elif '-' not in navps:
        return float(navps.replace(',',''))

In [15]:
calculate_eps('2', '12.00')

6.0

In [16]:
html_text = uReq.get('https://www.dsebd.org/latest_share_price_scroll_l.php').text

In [17]:
soup = BeautifulSoup(html_text,'lxml')

In [18]:
stocks_tcode_url = soup.find('table', class_ = 'table table-bordered background-white shares-table fixedHeader').find_all('a')

In [19]:
list_temp_data_hold = list()

In [20]:
stock_tcode_url_dict = dict()

In [21]:
for stock in stocks_tcode_url:
    stock_tcode_url_dict[f'{stock.text.strip()}'] = 'https://www.dsebd.org/' + stock['href']

In [22]:
corporate_disclosures = open('corporate_disclosures.csv', mode = 'w')

In [23]:
trading_codes = open('trading_codes.csv', mode = 'w')

In [24]:
with corporate_disclosures, trading_codes:
    
    write_disclosures = csv.writer(corporate_disclosures)
    write_tcode = csv.writer(trading_codes)
    
    for stock_tcode, url in sorted(stock_tcode_url_dict.items()):
    
        try:
            #browse and download from the website, store the extracted lxml parsed page on soup_company.
            html_text_company = uReq.get(url).text
            soup_company = BeautifulSoup(html_text_company,'lxml')
        
            #group all the table from stock's page and extract disclosures 
            company_table_data = soup_company.find_all('table', class_ = 'table table-bordered background-white')
        
            
            if '-' not in company_table_data[0].find('tr').find('td').text:
                trading_price = company_table_data[0].find('tr').find('td').text.replace(',','') #last trading Price
            elif '-' in company_table_data[0].find('tr').find('td').text:
                trading_price = company_table_data[0].find('tr').find('td').text
        
            sector = company_table_data[1].find_all('tr', class_='alt')[1].find_all('td')[1].text #sector
        
            pe_quarter = company_table_data[4].find('tr', class_ = 'alt').find_all('td')[-1].text #P/E: Quarterly report
            eps_quarter = calculate_eps(pe_quarter, trading_price)
        
            pe_annual = company_table_data[5].find('tr', class_='alt').find_all('td')[-1].text #P/E audited report
            eps_annual = calculate_eps(pe_annual, trading_price) 
        
            last_dividend = company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip() #dividend return
            dividend_paid = dividend_return(last_dividend, trading_price)
    
            navps = company_table_data[6].find_all('tr')[-1].find_all('td')[-6].text.strip() #NAVPS
            navps_obtained = calculate_navps(navps)

            audit_year = company_table_data[7].find_all('tr')[-1].td.text #year of last audited financial report
        
            market_category = company_table_data[9].find('tr').find_next_sibling('tr').find('td').find_next_sibling('td').text.strip() #category
        
        
            #list_temp_data_hold.append([stock_tcode, trading_price, sector, market_category, recent_dividend, pe_quarter, pe_annual, navps, audit_year])
            write_disclosures.writerow([stock_tcode, trading_price, sector, market_category, dividend_paid, eps_quarter, eps_annual, navps_obtained, audit_year])
    
            write_tcode.writerow([stock_tcode])
        
        except IndexError:
            #browse and download from the website, store the extracted lxml parsed page on soup_company.
            html_text_company = uReq.get(url).text
            soup_company = BeautifulSoup(html_text_company,'lxml')
        
            #group all the table from stock's page and extract disclosures 
            company_table_data = soup_company.find_all('table', class_ = 'table table-bordered background-white')
        
            
            if '-' not in company_table_data[0].find('tr').find('td').text:
                trading_price = company_table_data[0].find('tr').find('td').text.replace(',','') #last trading Price
            elif '-' in company_table_data[0].find('tr').find('td').text:
                trading_price = company_table_data[0].find('tr').find('td').text
        
            sector = company_table_data[1].find_all('tr', class_='alt')[1].find_all('td')[1].text #sector
        
            pe_quarter = company_table_data[4].find('tr', class_ = 'alt').find_all('td')[-1].text #P/E: Quarterly report
            eps_quarter = calculate_eps(pe_quarter, trading_price)
        
            pe_annual = company_table_data[5].find('tr', class_='alt').find_all('td')[-1].text #P/E audited report
            eps_annual = calculate_eps(pe_annual, trading_price) 
        
            last_dividend = company_table_data[7].find_all('tr')[-1].find_all('td')[-2].text.strip() #dividend return
            dividend_paid = dividend_return(last_dividend, trading_price)
    
            navps = company_table_data[6].find_all('tr')[-1].find_all('td')[-6].text.strip() #NAVPS
            navps_obtained = calculate_navps(navps)

            audit_year = company_table_data[7].find_all('tr')[-1].td.text #year of last audited financial report
        
            market_category = company_table_data[9].find('tr').find_next_sibling('tr').find('td').find_next_sibling('td').text.strip() #category
        
        
            #list_temp_data_hold.append([stock_tcode, trading_price, sector, market_category, recent_dividend, pe_quarter, pe_annual, navps, audit_year])
            write_disclosures.writerow([stock_tcode, trading_price, sector, market_category, dividend_paid, eps_quarter, eps_annual, navps_obtained, audit_year])
    
            write_tcode.writerow([stock_tcode])
        
        else:
            continue

In [25]:
'502' in soup_company.text 

False

In [26]:
len(company_table_data) == 0

False

In [27]:
a = [1, 2]

In [28]:
a[2]

IndexError: list index out of range

In [ ]:
url

In [ ]:
dividend_return('10.00 B', '27.2')

In [ ]:
re.findall('\d*\.?\d+', '12.22')